In [1]:
from __future__ import print_function
import numpy as np
import matplotlib.pyplot as plt
import glob
import pandas as pd
from keras.layers.advanced_activations import PReLU
from keras.layers import LSTM
from keras.layers import merge
from keras.preprocessing import sequence
from keras.models import Model
from keras.layers import Dense, Activation, Embedding, Flatten, Input, Dropout, Convolution1D, GlobalMaxPooling1D, merge,LSTM

Using TensorFlow backend.


In [2]:
names=glob.glob('Data/*.txt')
data = []
maxlen=0
for f in names:
    df=pd.read_csv(f,delimiter= ' ', header=None).values #load all the files into an numpy ndarray with 2d (lines of file, pairs) separated by ' '. 
    df=df[:,0:2]
    
    if maxlen <  df.shape[0]:
        maxlen=df.shape[0]    #find the max length of all the files in order to do later zero padding to the same length
    data.append(df)#append all these numpy ndarray  into a list
data=np.array(data)#parshe the list into numpy ndarray type.

print('Choose training & testing set and make Zero Padding')
#########
p=0.80 ##the proportion of training set (0.8 means --> 80% training set and 20% testing set)
#########
X_train = sequence.pad_sequences(data[:int(len(data)*p)], maxlen=maxlen) #zero padding to maxlen of pairs
                                          
X_test = sequence.pad_sequences(data[(int(len(data)*p)+1):], maxlen=maxlen) #zero padding to maxlen of pairs

y_train=np.ones(len(X_train)) # arry with targets ones that declere that we have causal directions
y_test=np.ones(len(X_test))
X_train_reverced = X_train[:,:,::-1] #we  reverse the pairs so as not to have causal direction

y_train_reverced = np.zeros(len(X_train)) #put targets zero that declare the lack of causal direction
merged_X_train= np.append(X_train,X_train_reverced,axis=0) #merge these data with causal directions and those without into one 
merged_y_train = np.append(y_train,y_train_reverced,axis=0) #merge the targets into one
X_train=merged_X_train
y_train=merged_y_train
X_test=X_test
y_test=y_test

print(X_train.shape)
print(y_train.shape)
print(X_test.shape)
print(y_test.shape)

Choose training & testing set and make Zero Padding
(144, 16382, 2)
(144,)
(17, 16382, 2)
(17,)


In [3]:
print('Build model...')
batch_size = 32

inputs=Input(shape=(X_train.shape[1:]),)


x = inputs

x=Convolution1D(nb_filter=64, filter_length=3, border_mode='same')(x)
x= Activation("relu")(x)
x=GlobalMaxPooling1D()(x)
#x=Dropout(0.1)(x)
x=Dense(64)(x)
x= Activation("relu")(x)
x=Dense(1)(x)
x= Activation("sigmoid")(x)
predictions = Activation("sigmoid")(x)

model = Model(input=inputs, output=predictions)
model.compile(loss='binary_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])
print("Finish model...")


Build model...
Finish model...


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:9: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(padding="same", kernel_size=3, filters=64)`
/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:19: UserWarning: Update your `Model` call to the Keras 2 API: `Model(outputs=Tensor("ac..., inputs=Tensor("in...)`


In [4]:
print('Train...')
model.fit(X_train, y_train, batch_size=batch_size, nb_epoch=15,
          validation_data=(X_test, y_test))

print('Evaluate...')
score, acc = model.evaluate(X_test, y_test,
                            batch_size=batch_size)
print('Test score:', score)
print('Test accuracy:', acc)

Train...


/usr/local/lib/python2.7/dist-packages/ipykernel/__main__.py:3: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  app.launch_new_instance()


Train on 144 samples, validate on 17 samples
Epoch 1/15
144/144 [==============================] - 33s - loss: 0.6934 - acc: 0.4722 - val_loss: 0.6370 - val_acc: 0.5882
Epoch 2/15
144/144 [==============================] - 33s - loss: 0.6939 - acc: 0.5347 - val_loss: 0.6468 - val_acc: 0.5882
Epoch 3/15
144/144 [==============================] - 33s - loss: 0.6938 - acc: 0.4931 - val_loss: 0.6510 - val_acc: 0.5882